# Gauss Thinning


In [ ]:
import igl
import meshplot as mp
import numpy as np
from gauss_thinning import *
from visualizations import *


## Load and Plot Data


In [ ]:
# v, f = igl.read_triangle_mesh("examples/einstein/input.off")
# v, f = igl.read_triangle_mesh("examples/bumpy/input.off")
# v, f = igl.read_triangle_mesh("examples/bunny/input.off")
v, f = igl.read_triangle_mesh("examples/face/input.off")


## Visualizing Surfaces and Pointclouds


In [ ]:
mp.plot(v, f, c=v[:, 0])
mp.plot(v, c=v[:, 0], shading={"point_size": 0.3})


## Gauss map of the input mesh


In [ ]:
# Calculating per face normals
nf = igl.per_face_normals(v, f, np.array([0.0, 0.0, 0.0]))
# Plotting face normals
mp.plot(v, f, c=nf)


In [ ]:
# Calculating per vertex normals
nv = igl.per_vertex_normals(v, f)
avg = igl.avg_edge_length(v, f)
# Plotting vertex normals as lines
pv = mp.plot(v, f, c=nv)
pv.add_lines(v, v + nv * avg)


In [ ]:
# Calculating the principal curvature, mean curvature and gassian curvature.
v1, v2, k1, k2 = igl.principal_curvature(v, f)
mean_curv = 0.5 * (k1 + k2)
gaussian_curv = k1 * k2
avg = igl.avg_edge_length(v, f)

# Plotting gaussian curvature + directions
pgc = mp.plot(v, f, c=gaussian_curv)
pgc.add_lines(v + v1 * avg / 3, v - v1 * avg / 3, shading={"line_color": "red"})
pgc.add_lines(v + v2 * avg / 3, v - v2 * avg / 3, shading={"line_color": "green"})


In [ ]:
# Plotting the gauss map
vs, fs = igl.read_triangle_mesh("examples/unit_sphere.obj")
vs = vs * 0.95
pgm = mp.plot(vs, fs, shading={"flat": False})
pgm.add_points(nf, shading={"point_size": 0.2})
ev, fe, ef = igl.edge_topology(v, f)
ef = ef[np.all(ef != -1, axis=1)]
pgm.add_lines(nf[ef[:, 0]], nf[ef[:, 1]], shading={"line_color": "red"})


## Visualizing the developability on the surface


In [ ]:
pd = mp.plot(v, f, c=developability_colors(v, f))


## Gauss Thinning


In [ ]:
v, f = igl.read_triangle_mesh("examples/bunny_small/input.off")
mp.plot(v, f, c=v[:, 0])
v_dash, _ = gauss_thinning(v, f, num_iterations=100, min_cone_angle=5, render=True)


In [ ]:
pd = mp.plot(v, f, c=developability_colors(v, f))
pd = mp.plot(v_dash, f, c=developability_colors(v_dash, f))


In [ ]:
epsilon = 1.5  # degrees

pd = mp.plot(v, f, c=angle_sum_colors(v, f, epsilon=epsilon))
pd = mp.plot(v_dash, f, c=angle_sum_colors(v_dash, f, epsilon=epsilon))


## Gauss Thinning with Gauss Image


In [ ]:
v, f = igl.read_triangle_mesh("examples/bunny_small/input.off")
plot = mp.plot(v, f, c=v[:, 0], return_plot=True)

gauss_offset = np.array([1, 0, 0])
gauss_scale = 0.4

n = igl.per_face_normals(v, f, np.array([0.0, 0.0, 0.0]))
vs, fs = igl.read_triangle_mesh("examples/unit_sphere.obj")
vs = vs * gauss_scale * 0.95 + gauss_offset
plot.add_mesh(vs, fs, shading={"flat": False})
plot.add_points(n * gauss_scale + gauss_offset, shading={"point_size": 0.1})
_, _, ef = igl.edge_topology(v, f)
ef = ef[np.all(ef != -1, axis=1)]
plot.add_lines(
    n[ef[:, 0]] * gauss_scale + gauss_offset,
    n[ef[:, 1]] * gauss_scale + gauss_offset,
    shading={"line_color": "red"},
)

v_dash, plot_dash = gauss_thinning(
    v, f, num_iterations=100, min_cone_angle=5, render=True
)

n = igl.per_face_normals(v_dash, f, np.array([0.0, 0.0, 0.0]))
plot_dash.add_mesh(vs, fs, shading={"flat": False})
plot_dash.add_points(n * gauss_scale + gauss_offset, shading={"point_size": 0.1})
_, _, ef = igl.edge_topology(v, f)
ef = ef[np.all(ef != -1, axis=1)]
plot_dash.add_lines(
    n[ef[:, 0]] * gauss_scale + gauss_offset,
    n[ef[:, 1]] * gauss_scale + gauss_offset,
    shading={"line_color": "red"},
)
